# Tutorial 02: Impulse Response Function (IRF) Analysis

**Estimated duration**: 90-120 minutes | **Level**: Intermediate

---

## Learning Objectives

By the end of this tutorial, you will be able to:

1. Understand the Moving Average (MA) representation of a VAR and how IRFs are derived
2. Compute orthogonalized IRFs using the **Cholesky decomposition** and explain the role of variable ordering
3. Compute **Generalized IRFs** (Pesaran-Shin) and understand when order-invariant IRFs are preferred
4. Compute and interpret **cumulative IRFs** as dynamic multipliers
5. Construct **bootstrap confidence intervals** and assess the statistical significance of impulse responses
6. Apply IRF analysis to real-world economic questions: monetary policy transmission and energy price pass-through

## Prerequisites

- **Notebook 01**: Introduction to Panel VAR (estimation, stability, lag selection)
- Basic knowledge of matrix algebra (Cholesky decomposition)
- Familiarity with VAR estimation and interpretation

## Outline

| Section | Topic | Duration |
|---------|-------|----------|
| 1 | IRF Theory and MA Representation | 20 min |
| 2 | Orthogonalized IRF -- Cholesky Decomposition | 25 min |
| 3 | Generalized IRF -- Pesaran-Shin | 25 min |
| 4 | Cumulative IRFs | 15 min |
| 5 | Bootstrap Confidence Intervals | 30 min |
| 6 | Economic Applications | 15 min |
| 7 | Diagnostics and Best Practices | 10 min |

## References

- Lutkepohl, H. (2005). *New Introduction to Multiple Time Series Analysis*. Springer.
- Pesaran, H. H., & Shin, Y. (1998). Generalized impulse response analysis in linear multivariate models. *Economics Letters*, 58(1), 17-29.
- Kilian, L. (1998). Small-sample confidence intervals for impulse response functions. *Review of Economics and Statistics*, 80(2), 218-230.
- Sims, C. A. (1980). Macroeconomics and reality. *Econometrica*, 48(1), 1-48.

---

## Setup

In [ ]:
# Standard library
import sys
import os
import warnings

# Scientific computing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
from scipy import linalg

# PanelBox VAR
from panelbox.var import PanelVARData, PanelVAR

# Tutorial utilities
sys.path.insert(0, os.path.abspath('..'))
from utils.visualization_helpers import (
    plot_irf_grid,
    plot_irf_comparison,
    set_academic_style,
)

# Configuration
set_academic_style()
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)
np.random.seed(42)

# Output directory for saved figures
FIGURE_DIR = '../outputs/figures/irfs'
os.makedirs(FIGURE_DIR, exist_ok=True)

print('Setup complete.')
print(f'NumPy version: {np.__version__}')
print(f'Pandas version: {pd.__version__}')

---

## Section 1: IRF Theory and the MA Representation (20 min)

### 1.1 From VAR to Moving Average Representation

A stable VAR(p) process can be written in **Moving Average (MA) form**:

$$
Y_t = \mu + \sum_{h=0}^{\infty} \Phi_h \, \varepsilon_{t-h}
$$

where:
- $Y_t$ is the $K \times 1$ vector of endogenous variables
- $\varepsilon_t$ is the $K \times 1$ vector of structural shocks
- $\Phi_h$ is the $K \times K$ matrix of impulse response coefficients at horizon $h$

The **impulse response function** $\Phi_h$ captures the effect on $Y_{t+h}$ of a one-time shock at time $t$.

### 1.2 Recursive Formula for $\Phi_h$

The MA coefficients are computed recursively from the VAR coefficient matrices:

$$
\Phi_0 = I_K
$$
$$
\Phi_h = \sum_{l=1}^{\min(h,p)} A_l \cdot \Phi_{h-l}, \quad h = 1, 2, \ldots
$$

where $A_1, A_2, \ldots, A_p$ are the VAR coefficient matrices.

### 1.3 The Identification Problem

The reduced-form VAR residuals $u_t$ are generally **correlated** across equations:

$$
E[u_t u_t'] = \Sigma \neq \text{diagonal}
$$

This means a shock to one variable simultaneously affects others. To give IRFs a **structural** interpretation, we need to **orthogonalize** the shocks, i.e., decompose correlated residuals into uncorrelated structural shocks.

This is the **identification problem**: how to map reduced-form innovations $u_t$ to structural shocks $\varepsilon_t$?

### 1.4 Why We Need Orthogonalization

Without orthogonalization:
- A "shock to GDP" also moves inflation (because $u_{\text{GDP}}$ and $u_{\text{inflation}}$ are correlated)
- We cannot isolate the **independent** effect of each variable
- Policy analysis requires knowing what happens when **only one** structural disturbance changes

Two main approaches:
1. **Cholesky decomposition** (recursive structure) -- depends on variable ordering
2. **Generalized IRFs** (Pesaran-Shin, 1998) -- order-invariant, but no structural interpretation

### 1.5 Load Data and Estimate VAR

In [ ]:
# Load the macroeconomic panel dataset
df_macro = pd.read_csv('../data/macro_panel.csv')

# Convert quarter strings to pandas PeriodIndex for proper temporal handling
df_macro['quarter'] = pd.PeriodIndex(df_macro['quarter'], freq='Q')

print(f'Dataset shape: {df_macro.shape}')
print(f'Countries: {df_macro["country"].nunique()}')
print(f'Quarters: {df_macro["quarter"].nunique()}')
print(f'\nColumns: {list(df_macro.columns)}')
print(f'\nQuarter format (first 3): {df_macro["quarter"].unique()[:3].tolist()}')
df_macro.head()

In [ ]:
# Prepare data for Panel VAR estimation
# We focus on 3 core macro variables for clarity
var_variables = ['interest_rate', 'inflation', 'gdp_growth']

var_data = PanelVARData(
    data=df_macro,
    endog_vars=var_variables,
    entity_col='country',
    time_col='quarter',
    lags=2,
)

print(f'Panel VAR data prepared:')
print(f'  Entities (N): {var_data.N}')
print(f'  Lags (p): {var_data.p}')
print(f'  Endogenous variables (K): {var_data.K}')
print(f'  Variables: {var_data.endog_vars}')

In [ ]:
# Estimate VAR(2) with fixed effects
model = PanelVAR(var_data)
results = model.fit()

print('VAR(2) estimated successfully.')
print(f'  Number of lags: {results.p}')
print(f'  Number of variables: {results.K}')
print(f'  Observations used: {results.n_obs}')
print(f'  Stability: {"STABLE" if results.is_stable() else "UNSTABLE"}')
print(f'  Max eigenvalue modulus: {results.max_eigenvalue_modulus:.4f}')

In [ ]:
# Examine the residual covariance matrix -- the key to understanding IRFs
Sigma = results.Sigma

print('Residual Covariance Matrix (Sigma):')
print('=' * 50)
sigma_df = pd.DataFrame(
    Sigma,
    index=results.endog_names,
    columns=results.endog_names
)
print(sigma_df.round(4))

# Compute residual correlation matrix
D_inv = np.diag(1.0 / np.sqrt(np.diag(Sigma)))
corr_matrix = D_inv @ Sigma @ D_inv

print('\nResidual Correlation Matrix:')
print('=' * 50)
corr_df = pd.DataFrame(
    corr_matrix,
    index=results.endog_names,
    columns=results.endog_names
)
print(corr_df.round(4))

print('\n--- Key Insight ---')
print('Off-diagonal correlations are NON-ZERO, meaning residuals are correlated.')
print('This is why we need orthogonalization to identify structural shocks.')

In [ ]:
# Visualize the correlation matrix
fig, ax = plt.subplots(figsize=(7, 5))
sns.heatmap(
    corr_df,
    annot=True,
    fmt='.3f',
    cmap='RdBu_r',
    center=0,
    vmin=-1,
    vmax=1,
    square=True,
    linewidths=0.5,
    ax=ax,
)
ax.set_title('Residual Correlation Matrix', fontsize=14, fontweight='bold')
fig.tight_layout()
fig.savefig(f'{FIGURE_DIR}/residual_correlation.png', dpi=150, bbox_inches='tight')
plt.show()

---

## Section 2: Orthogonalized IRF -- Cholesky Decomposition (25 min)

### 2.1 Cholesky Decomposition Theory

The Cholesky decomposition factors the positive-definite covariance matrix $\Sigma$ as:

$$
\Sigma = P \cdot P'
$$

where $P$ is a **lower-triangular** matrix. This imposes a **recursive causal ordering**:
- The first variable responds **only** to its own structural shock contemporaneously
- The second variable responds to shocks in the first and second variables
- And so on...

The orthogonalized impulse responses are:

$$
\Theta_0 = P, \quad \Theta_h = \Phi_h \cdot P
$$

### 2.2 Role of Variable Ordering

The Cholesky decomposition is **not unique** -- it depends on the ordering of variables. This reflects an economic assumption about **contemporaneous causality**:

| Ordering Position | Interpretation |
|:-:|:---|
| First | Most exogenous: not affected contemporaneously by other variables |
| Last | Most endogenous: affected by all variables contemporaneously |

### 2.3 Economic Justification for Our Ordering

We use: **interest_rate** $\to$ **inflation** $\to$ **gdp_growth**

Rationale:
- **Interest rate first**: Central banks set policy rates based on last period's information (predetermined within the quarter)
- **Inflation second**: Prices adjust before real activity within the quarter
- **GDP growth last**: Real output responds to both monetary policy and price signals within the quarter

This is a standard ordering in the monetary policy VAR literature (Christiano, Eichenbaum, Evans, 1999).

In [ ]:
# Demonstrate Cholesky decomposition manually
P = linalg.cholesky(Sigma, lower=True)

print('Cholesky Factor P (lower triangular):')
print('=' * 50)
P_df = pd.DataFrame(
    P,
    index=results.endog_names,
    columns=results.endog_names
)
print(P_df.round(4))

print('\nVerification: P @ P.T = Sigma?')
print(f'Max absolute error: {np.max(np.abs(P @ P.T - Sigma)):.2e}')

print('\n--- Key Insight ---')
print('P is lower triangular: the first variable (interest_rate) is not affected')
print('contemporaneously by shocks to other variables. This is the recursive assumption.')

### 2.4 Compute Cholesky IRFs

In [ ]:
# Compute orthogonalized IRFs using Cholesky decomposition
irf_cholesky = results.irf(
    periods=20,
    method='cholesky',
    order=['interest_rate', 'inflation', 'gdp_growth'],
    cumulative=False,
)

print(f'IRF Result: {irf_cholesky}')
print(f'\nIRF matrix shape: {irf_cholesky.irf_matrix.shape}  (periods+1, K, K)')
print(f'Variable names: {irf_cholesky.var_names}')
print(f'Method: {irf_cholesky.method}')
print(f'Cumulative: {irf_cholesky.cumulative}')
print(f'K = {irf_cholesky.K}, periods = {irf_cholesky.periods}')

In [ ]:
# Display the summary table at selected horizons
print(irf_cholesky.summary(horizons=[0, 1, 4, 8, 20]))

### 2.5 Visualize Cholesky IRFs

In [ ]:
# Use the built-in plot method: all responses to an interest_rate shock
fig = irf_cholesky.plot(
    impulse='interest_rate',
    ci=True,
    backend='matplotlib',
    show=False,
)
if fig is not None:
    fig.savefig(f'{FIGURE_DIR}/cholesky_interest_rate_shock.png', dpi=150, bbox_inches='tight')
    plt.show()

In [ ]:
# Use our utility function for a full K*K grid
fig_grid = plot_irf_grid(
    irf_cholesky,
    save_path=f'{FIGURE_DIR}/cholesky_irf_grid.png',
)
plt.show()

In [ ]:
# Manually plot a specific IRF: GDP response to interest rate shock
# Access with: irf['response_var', 'impulse_var']
gdp_response = irf_cholesky['gdp_growth', 'interest_rate']
horizons = np.arange(len(gdp_response))

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(horizons, gdp_response, color='#2166ac', linewidth=2.2, marker='o', markersize=4)
ax.axhline(y=0, color='black', linewidth=0.8, linestyle='--', alpha=0.5)
ax.fill_between(horizons, 0, gdp_response, alpha=0.15, color='#2166ac')

# Mark peak response
peak_h = np.argmin(gdp_response)  # Expect negative response
peak_val = gdp_response[peak_h]
ax.annotate(
    f'Peak: {peak_val:.4f} at h={peak_h}',
    xy=(peak_h, peak_val),
    xytext=(peak_h + 2, peak_val - 0.02),
    fontsize=10,
    arrowprops=dict(arrowstyle='->', color='gray'),
    bbox=dict(boxstyle='round,pad=0.3', facecolor='lightyellow', edgecolor='gray'),
)

ax.set_xlabel('Horizon (quarters)', fontsize=12)
ax.set_ylabel('Response of GDP Growth', fontsize=12)
ax.set_title('GDP Growth Response to Interest Rate Shock (Cholesky)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
fig.tight_layout()
fig.savefig(f'{FIGURE_DIR}/gdp_response_to_ir.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\nInterpretation:')
print(f'  Impact effect (h=0): {gdp_response[0]:.4f}')
print(f'  Peak response: {peak_val:.4f} at horizon h={peak_h}')
print(f'  Long-run response (h=20): {gdp_response[-1]:.4f}')

In [ ]:
# Convert to DataFrame for tabular analysis
df_irf = irf_cholesky.to_dataframe(impulse='interest_rate')
print('IRF as DataFrame (interest_rate shock):')
print(df_irf.head(11).to_string(index=False))

### 2.6 Interpreting the Results

**Reading the IRF (GDP response to interest rate shock):**

- **h=0**: The contemporaneous impact. Under our Cholesky ordering, interest rates are ordered first, so a shock to interest rates can affect GDP growth immediately.
- **Peak effect**: The maximum (absolute) response, often delayed by 2-4 quarters -- this is the "monetary policy transmission lag."
- **Convergence**: In a stable system, responses decay toward zero as $h \to \infty$.

**Economic interpretation**: A positive interest rate shock (contractionary monetary policy) should reduce GDP growth, consistent with standard macroeconomic theory.

---

## Section 3: Generalized IRF -- Pesaran-Shin (25 min)

### 3.1 Motivation: Order-Invariance

The Cholesky decomposition requires a specific variable ordering, which is often **controversial**. Pesaran and Shin (1998) proposed **Generalized IRFs (GIRFs)** that are **invariant to the ordering** of variables.

### 3.2 The GIRF Formula

For a one-standard-deviation shock to variable $j$:

$$
GIRF_j(h) = \frac{1}{\sqrt{\sigma_{jj}}} \Phi_h \cdot \Sigma \cdot e_j
$$

where:
- $\Phi_h$ is the non-orthogonalized MA coefficient at horizon $h$ (with $\Phi_0 = I_K$)
- $\Sigma$ is the residual covariance matrix
- $e_j$ is the $j$-th unit vector
- $\sigma_{jj}$ is the $j$-th diagonal element of $\Sigma$

**Key property**: The GIRF does not impose any recursive structure; it asks: *given the historically observed correlation structure, what is the typical response to a shock in variable $j$?*

In [ ]:
# Compute Generalized IRFs
irf_generalized = results.irf(
    periods=20,
    method='generalized',
)

print(f'Generalized IRF: {irf_generalized}')
print(f'Method: {irf_generalized.method}')

In [ ]:
# Display the generalized IRF summary
print(irf_generalized.summary(horizons=[0, 1, 4, 8, 20]))

### 3.3 Compare Cholesky vs. Generalized IRFs

In [ ]:
# Side-by-side comparison using our utility function
fig_cmp = plot_irf_comparison(
    irf_cholesky,
    irf_generalized,
    impulse='interest_rate',
    response='gdp_growth',
    save_path=f'{FIGURE_DIR}/cholesky_vs_generalized_gdp.png',
)
plt.show()

In [ ]:
# Detailed comparison: overlay both methods on same axes for each response variable
fig, axes = plt.subplots(1, 3, figsize=(16, 4.5))

for idx, response_var in enumerate(irf_cholesky.var_names):
    ax = axes[idx]
    h = np.arange(irf_cholesky.periods + 1)

    # Cholesky
    chol_vals = irf_cholesky[response_var, 'interest_rate']
    ax.plot(h, chol_vals, color='#2166ac', linewidth=2, label='Cholesky', linestyle='-')

    # Generalized
    gen_vals = irf_generalized[response_var, 'interest_rate']
    ax.plot(h, gen_vals, color='#b2182b', linewidth=2, label='Generalized', linestyle='--')

    ax.axhline(y=0, color='black', linewidth=0.8, linestyle='--', alpha=0.5)
    ax.set_title(f'Response: {response_var}', fontsize=12, fontweight='bold')
    ax.set_xlabel('Horizon', fontsize=11)
    if idx == 0:
        ax.set_ylabel('Response', fontsize=11)
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)

fig.suptitle('Interest Rate Shock: Cholesky vs. Generalized IRFs', fontsize=14, fontweight='bold', y=1.02)
fig.tight_layout()
fig.savefig(f'{FIGURE_DIR}/cholesky_vs_generalized_overlay.png', dpi=150, bbox_inches='tight')
plt.show()

### 3.4 When to Use Which Method?

| Feature | Cholesky (Orthogonalized) | Generalized (Pesaran-Shin) |
|:--------|:-------------------------|:---------------------------|
| **Ordering dependence** | Yes -- results change with ordering | No -- invariant to ordering |
| **Structural interpretation** | Yes -- causal chain assumption | No -- purely statistical |
| **Orthogonal shocks** | Yes -- shocks are uncorrelated | No -- shocks remain correlated |
| **Use when...** | You have a defensible causal ordering | Ordering is unclear or controversial |
| **Common in...** | Monetary policy VARs (Sims, Christiano et al.) | International macro, financial contagion |
| **Decomposition** | IRFs sum to explain forecast error variance | GIRFs do not partition variance |
| **Diagonal at h=0** | $\Theta_0$ is lower triangular | $GIRF_0$ is generally full |

**Rule of thumb**: If you can justify a causal ordering from economic theory, use Cholesky. If not, use Generalized IRFs.

### 3.5 Demonstrating Order Invariance

In [ ]:
# Demonstrate: Generalized IRFs are the same regardless of variable ordering
# Re-estimate VAR with reversed variable order
reversed_vars = ['gdp_growth', 'inflation', 'interest_rate']  # Reversed ordering

var_data_rev = PanelVARData(
    data=df_macro,
    endog_vars=reversed_vars,
    entity_col='country',
    time_col='quarter',
    lags=2,
)
model_rev = PanelVAR(var_data_rev)
results_rev = model_rev.fit()

# Generalized IRF with reversed ordering
irf_gen_reversed = results_rev.irf(
    periods=20,
    method='generalized',
)

# Compare: GDP response to interest rate shock
# Original ordering
girf_original = irf_generalized['gdp_growth', 'interest_rate']

# Reversed ordering (same variable names, different model ordering)
girf_reversed = irf_gen_reversed['gdp_growth', 'interest_rate']

# Check if they are approximately the same
max_diff = np.max(np.abs(girf_original - girf_reversed))

print('Generalized IRF -- Order Invariance Test')
print('=' * 55)
print(f'Original ordering:  {irf_generalized.var_names}')
print(f'Reversed ordering:  {irf_gen_reversed.var_names}')
print(f'\nMax absolute difference in GIRF(gdp_growth <- interest_rate): {max_diff:.2e}')
print(f'GIRFs are identical (within numerical precision): {max_diff < 1e-10}')

# Now demonstrate that Cholesky IRFs DIFFER with ordering
irf_chol_reversed = results_rev.irf(
    periods=20,
    method='cholesky',
    order=['gdp_growth', 'inflation', 'interest_rate'],
)

chol_original = irf_cholesky['gdp_growth', 'interest_rate']
chol_reversed = irf_chol_reversed['gdp_growth', 'interest_rate']
max_diff_chol = np.max(np.abs(chol_original - chol_reversed))

print(f'\nMax difference in Cholesky IRF with reversed ordering: {max_diff_chol:.4f}')
print(f'Cholesky IRFs differ with ordering: {max_diff_chol > 0.001}')

print('\n--- Key Insight ---')
print('Generalized IRFs are invariant to variable ordering.')
print('Cholesky IRFs depend on the assumed causal ordering.')

---

## Section 4: Cumulative IRFs (15 min)

### 4.1 Theory: Cumulative Impulse Responses

The **cumulative IRF** at horizon $h$ represents the total accumulated effect of a shock:

$$
\Psi_h = \sum_{s=0}^{h} \Phi_s
$$

**When is this useful?**
- When variables are in **growth rates** (e.g., GDP growth): the cumulative IRF gives the effect on the **level** of GDP
- Computing **long-run multipliers**: $\Psi_\infty = \sum_{s=0}^{\infty} \Phi_s$
- Comparing total impact across different shocks

**Dynamic multiplier at horizon $h$**: The cumulative IRF value $\Psi_h$ tells us the total change in the response variable from period 0 to period $h$ due to a one-time shock.

In [ ]:
# Compute cumulative IRFs
irf_cumulative = results.irf(
    periods=20,
    method='cholesky',
    order=['interest_rate', 'inflation', 'gdp_growth'],
    cumulative=True,
)

print(f'Cumulative IRF: {irf_cumulative}')
print(f'Cumulative: {irf_cumulative.cumulative}')

In [ ]:
# Dynamic multipliers at selected horizons
print('Dynamic Multipliers: GDP Growth Response to Interest Rate Shock')
print('=' * 65)
print(f'{"Horizon":>10} {"Level IRF":>15} {"Cumulative IRF":>18} {"Interpretation":>25}')
print('-' * 65)

for h in [0, 1, 4, 8, 12, 20]:
    level_val = irf_cholesky['gdp_growth', 'interest_rate'][h]
    cum_val = irf_cumulative['gdp_growth', 'interest_rate'][h]
    if h == 0:
        interp = 'Impact effect'
    elif h <= 4:
        interp = 'Short-run multiplier'
    elif h <= 12:
        interp = 'Medium-run multiplier'
    else:
        interp = 'Long-run multiplier'
    print(f'{h:>10} {level_val:>15.6f} {cum_val:>18.6f} {interp:>25}')

print('\n--- Key Insight ---')
print('The cumulative IRF at h=20 approximates the long-run multiplier.')
print('For GDP growth, this represents the total accumulated effect on the GDP level.')

In [ ]:
# Side-by-side: Level IRF vs. Cumulative IRF
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
h = np.arange(irf_cholesky.periods + 1)

# Level IRF
level_vals = irf_cholesky['gdp_growth', 'interest_rate']
axes[0].plot(h, level_vals, color='#2166ac', linewidth=2, marker='o', markersize=3)
axes[0].axhline(y=0, color='black', linewidth=0.8, linestyle='--', alpha=0.5)
axes[0].fill_between(h, 0, level_vals, alpha=0.15, color='#2166ac')
axes[0].set_title('Level IRF (period-by-period effect)', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Horizon (quarters)', fontsize=11)
axes[0].set_ylabel('Response of GDP Growth', fontsize=11)
axes[0].grid(True, alpha=0.3)

# Cumulative IRF
cum_vals = irf_cumulative['gdp_growth', 'interest_rate']
axes[1].plot(h, cum_vals, color='#b2182b', linewidth=2, marker='s', markersize=3)
axes[1].axhline(y=0, color='black', linewidth=0.8, linestyle='--', alpha=0.5)
axes[1].fill_between(h, 0, cum_vals, alpha=0.15, color='#b2182b')
axes[1].set_title('Cumulative IRF (accumulated effect)', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Horizon (quarters)', fontsize=11)
axes[1].set_ylabel('Cumulative Response', fontsize=11)
axes[1].grid(True, alpha=0.3)

fig.suptitle(
    'GDP Growth Response to Interest Rate Shock: Level vs. Cumulative',
    fontsize=14, fontweight='bold', y=1.03,
)
fig.tight_layout()
fig.savefig(f'{FIGURE_DIR}/level_vs_cumulative_irf.png', dpi=150, bbox_inches='tight')
plt.show()

---

## Section 5: Bootstrap Confidence Intervals (30 min)

### 5.1 Why Bootstrap?

Point estimates of IRFs are only part of the story. We need **confidence intervals** to:
- Assess **statistical significance** (does the response differ from zero?)
- Quantify **estimation uncertainty** at each horizon
- Make reliable policy conclusions

### 5.2 The Bootstrap Procedure

For Panel VAR IRFs, the bootstrap works as follows:

1. **Estimate** the VAR and obtain coefficient matrices $\hat{A}_1, \ldots, \hat{A}_p$ and residuals $\hat{u}_t$
2. **Resample** residuals with replacement: draw $\hat{u}^*_t$ from the empirical distribution of $\hat{u}_t$
3. **Simulate** new data using the estimated coefficients and resampled residuals
4. **Re-estimate** the VAR on the simulated data to get $\hat{A}^*_1, \ldots, \hat{A}^*_p$
5. **Compute** IRFs from the bootstrap estimates
6. **Repeat** $B$ times to build a distribution of IRFs
7. **Construct** confidence intervals from the percentiles of the bootstrap distribution

This method was proposed by Kilian (1998) and is standard in the applied VAR literature.

In [ ]:
# Compute IRFs with bootstrap confidence intervals
irf_boot = results.irf(
    periods=20,
    method='cholesky',
    order=['interest_rate', 'inflation', 'gdp_growth'],
    cumulative=False,
    ci_method='bootstrap',
    n_bootstrap=200,
    ci_level=0.95,
    seed=42,
    verbose=False,
)

print(f'Bootstrap IRF: {irf_boot}')
print(f'CI lower shape: {irf_boot.ci_lower.shape}')
print(f'CI upper shape: {irf_boot.ci_upper.shape}')
print(f'CI level: {irf_boot.ci_level}')

### 5.3 Visualize IRFs with Confidence Intervals

In [ ]:
# Built-in plot with confidence bands
fig = irf_boot.plot(
    impulse='interest_rate',
    ci=True,
    backend='matplotlib',
    show=False,
)
if fig is not None:
    fig.savefig(f'{FIGURE_DIR}/bootstrap_ci_interest_rate.png', dpi=150, bbox_inches='tight')
    plt.show()

In [ ]:
# Manually plot one IRF with fill_between for the confidence interval
response_var = 'gdp_growth'
impulse_var = 'interest_rate'

# Extract point estimate
irf_values = irf_boot[response_var, impulse_var]
horizons = np.arange(len(irf_values))

# Extract CI using index positions
response_idx = irf_boot.var_names.index(response_var)
impulse_idx = irf_boot.var_names.index(impulse_var)
lower = irf_boot.ci_lower[:, response_idx, impulse_idx]
upper = irf_boot.ci_upper[:, response_idx, impulse_idx]

fig, ax = plt.subplots(figsize=(10, 6))

# Confidence band
ax.fill_between(
    horizons, lower, upper,
    alpha=0.2, color='#2166ac', label='95% Bootstrap CI'
)

# Point estimate
ax.plot(
    horizons, irf_values,
    color='#2166ac', linewidth=2.2, label='Point Estimate',
    marker='o', markersize=3,
)

# CI boundaries
ax.plot(horizons, lower, color='#2166ac', linewidth=0.8, linestyle=':', alpha=0.6)
ax.plot(horizons, upper, color='#2166ac', linewidth=0.8, linestyle=':', alpha=0.6)

# Zero reference line
ax.axhline(y=0, color='black', linewidth=0.8, linestyle='--', alpha=0.5)

# Shade regions where CI excludes zero (statistically significant)
significant = (upper < 0) | (lower > 0)
for h_idx in range(len(horizons)):
    if significant[h_idx]:
        ax.axvspan(h_idx - 0.3, h_idx + 0.3, alpha=0.08, color='green')

ax.set_xlabel('Horizon (quarters)', fontsize=12)
ax.set_ylabel(f'Response of {response_var}', fontsize=12)
ax.set_title(
    f'{response_var} Response to {impulse_var} Shock (with 95% Bootstrap CI)',
    fontsize=14, fontweight='bold'
)
ax.legend(fontsize=10, loc='best')
ax.grid(True, alpha=0.3)
fig.tight_layout()
fig.savefig(f'{FIGURE_DIR}/manual_bootstrap_ci.png', dpi=150, bbox_inches='tight')
plt.show()

# Report significance
print('\nStatistical Significance by Horizon:')
print('=' * 55)
print(f'{"Horizon":>8} {"Point Est.":>12} {"CI Lower":>12} {"CI Upper":>12} {"Significant?":>14}')
print('-' * 55)
for h_val in [0, 1, 2, 4, 8, 12, 16, 20]:
    sig_label = 'YES' if significant[h_val] else 'no'
    print(f'{h_val:>8} {irf_values[h_val]:>12.4f} {lower[h_val]:>12.4f} {upper[h_val]:>12.4f} {sig_label:>14}')

### 5.4 Interpreting Significance

An IRF is **statistically significant** at horizon $h$ if the confidence interval **does not include zero**:

- If both CI bounds are **negative**: significant negative response
- If both CI bounds are **positive**: significant positive response
- If the CI **spans zero**: response is not statistically distinguishable from zero at the chosen confidence level

**Caution**: Wide confidence intervals at longer horizons are typical -- estimation uncertainty accumulates over the forecast horizon.

### 5.5 Sensitivity: B=100 vs. B=200

In [ ]:
# Compare bootstrap with different numbers of replications
irf_b100 = results.irf(
    periods=20,
    method='cholesky',
    order=['interest_rate', 'inflation', 'gdp_growth'],
    ci_method='bootstrap',
    n_bootstrap=100,
    ci_level=0.95,
    seed=42,
    verbose=False,
)

# Compare CI widths
resp_idx = irf_boot.var_names.index('gdp_growth')
imp_idx = irf_boot.var_names.index('interest_rate')

width_b200 = irf_boot.ci_upper[:, resp_idx, imp_idx] - irf_boot.ci_lower[:, resp_idx, imp_idx]
width_b100 = irf_b100.ci_upper[:, resp_idx, imp_idx] - irf_b100.ci_lower[:, resp_idx, imp_idx]

fig, ax = plt.subplots(figsize=(10, 5))
h = np.arange(irf_boot.periods + 1)

ax.plot(h, width_b100, color='#e66101', linewidth=2, label='B=100', marker='^', markersize=4)
ax.plot(h, width_b200, color='#5e3c99', linewidth=2, label='B=200', marker='o', markersize=4)

ax.set_xlabel('Horizon (quarters)', fontsize=12)
ax.set_ylabel('CI Width (upper - lower)', fontsize=12)
ax.set_title(
    'Bootstrap CI Width Sensitivity: B=100 vs. B=200\n(GDP Growth <- Interest Rate)',
    fontsize=13, fontweight='bold',
)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
fig.tight_layout()
fig.savefig(f'{FIGURE_DIR}/bootstrap_sensitivity.png', dpi=150, bbox_inches='tight')
plt.show()

print('\nCI Width Comparison (selected horizons):')
print(f'{"Horizon":>8} {"Width B=100":>14} {"Width B=200":>14} {"Diff (%)":>12}')
for h_val in [0, 4, 8, 12, 20]:
    pct_diff = 100 * (width_b200[h_val] - width_b100[h_val]) / width_b100[h_val] if width_b100[h_val] != 0 else 0
    print(f'{h_val:>8} {width_b100[h_val]:>14.4f} {width_b200[h_val]:>14.4f} {pct_diff:>11.1f}%')

print('\n--- Key Insight ---')
print('More bootstrap replications produce more stable CI estimates.')
print('For published work, B=500-1000 is recommended; B=200 is acceptable for exploration.')

---

## Section 6: Economic Applications (15 min)

### 6.1 Application 1: Monetary Policy Transmission

We analyze the full monetary transmission mechanism using our macro panel:
- How does a contractionary monetary policy shock (interest rate increase) propagate through the economy?
- What are the magnitudes and timing of the effects on inflation and GDP growth?

In [ ]:
# Monetary policy transmission: comprehensive IRF analysis
# Use bootstrap CIs for inference
irf_monetary = results.irf(
    periods=20,
    method='cholesky',
    order=['interest_rate', 'inflation', 'gdp_growth'],
    cumulative=False,
    ci_method='bootstrap',
    n_bootstrap=200,
    ci_level=0.95,
    seed=42,
    verbose=False,
)

# Analyze responses of each variable to interest rate shock
fig, axes = plt.subplots(1, 3, figsize=(16, 5))
h = np.arange(irf_monetary.periods + 1)

titles = {
    'interest_rate': 'Interest Rate (own response)',
    'inflation': 'Inflation (price puzzle?)',
    'gdp_growth': 'GDP Growth (IS curve)',
}
colors = ['#2166ac', '#d6604d', '#4dac26']

for idx, (var, title) in enumerate(titles.items()):
    ax = axes[idx]
    irf_vals = irf_monetary[var, 'interest_rate']

    # CI
    r_idx = irf_monetary.var_names.index(var)
    i_idx = irf_monetary.var_names.index('interest_rate')
    ci_lo = irf_monetary.ci_lower[:, r_idx, i_idx]
    ci_hi = irf_monetary.ci_upper[:, r_idx, i_idx]

    ax.fill_between(h, ci_lo, ci_hi, alpha=0.2, color=colors[idx])
    ax.plot(h, irf_vals, color=colors[idx], linewidth=2)
    ax.axhline(y=0, color='black', linewidth=0.8, linestyle='--', alpha=0.5)
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.set_xlabel('Horizon (quarters)', fontsize=11)
    if idx == 0:
        ax.set_ylabel('Response', fontsize=11)
    ax.grid(True, alpha=0.3)

fig.suptitle(
    'Monetary Policy Transmission: Responses to Interest Rate Shock',
    fontsize=14, fontweight='bold', y=1.03,
)
fig.tight_layout()
fig.savefig(f'{FIGURE_DIR}/monetary_transmission.png', dpi=150, bbox_inches='tight')
plt.show()

# Summary statistics
print('\nMonetary Policy Transmission Summary')
print('=' * 60)
for var in ['inflation', 'gdp_growth']:
    vals = irf_monetary[var, 'interest_rate']
    peak_idx = np.argmax(np.abs(vals))
    print(f'\n  {var}:')
    print(f'    Impact effect (h=0):  {vals[0]:.4f}')
    print(f'    Peak effect:          {vals[peak_idx]:.4f} at h={peak_idx}')
    print(f'    Half-life:            ~{np.argmin(np.abs(vals[peak_idx:] - vals[peak_idx]/2)) + peak_idx} quarters')
    print(f'    Long-run (h=20):      {vals[-1]:.4f}')

### 6.2 Application 2: Energy Price Transmission

In [ ]:
# Load energy panel dataset
df_energy = pd.read_csv('../data/energy_panel.csv')

# Convert quarter strings to pandas PeriodIndex
df_energy['quarter'] = pd.PeriodIndex(df_energy['quarter'], freq='Q')

print(f'Energy panel shape: {df_energy.shape}')
print(f'Countries: {df_energy["country"].nunique()}')
print(f'Variables: {list(df_energy.columns)}')
df_energy.head()

In [ ]:
# Estimate Panel VAR on energy prices
energy_vars = ['oil_price', 'gas_price', 'electricity_price']

energy_data = PanelVARData(
    data=df_energy,
    endog_vars=energy_vars,
    entity_col='country',
    time_col='quarter',
    lags=2,
)

energy_model = PanelVAR(energy_data)
energy_results = energy_model.fit()

print(f'Energy VAR(2) estimated: K={energy_results.K}, N={energy_results.N}')
print(f'Stability: {"STABLE" if energy_results.is_stable() else "UNSTABLE"}')
print(f'Max eigenvalue modulus: {energy_results.max_eigenvalue_modulus:.4f}')

In [ ]:
# IRFs for energy price transmission (oil -> gas -> electricity)
irf_energy = energy_results.irf(
    periods=20,
    method='cholesky',
    order=['oil_price', 'gas_price', 'electricity_price'],
    ci_method='bootstrap',
    n_bootstrap=200,
    ci_level=0.95,
    seed=42,
    verbose=False,
)

# Plot: downstream price responses to oil price shock
fig, axes = plt.subplots(1, 3, figsize=(16, 5))
h = np.arange(irf_energy.periods + 1)

energy_titles = {
    'oil_price': 'Oil Price (own shock)',
    'gas_price': 'Gas Price (upstream spillover)',
    'electricity_price': 'Electricity Price (downstream)',
}
energy_colors = ['#1b7837', '#762a83', '#e66101']

for idx, (var, title) in enumerate(energy_titles.items()):
    ax = axes[idx]
    vals = irf_energy[var, 'oil_price']

    # CI
    r_idx = irf_energy.var_names.index(var)
    i_idx = irf_energy.var_names.index('oil_price')
    ci_lo = irf_energy.ci_lower[:, r_idx, i_idx]
    ci_hi = irf_energy.ci_upper[:, r_idx, i_idx]

    ax.fill_between(h, ci_lo, ci_hi, alpha=0.2, color=energy_colors[idx])
    ax.plot(h, vals, color=energy_colors[idx], linewidth=2)
    ax.axhline(y=0, color='black', linewidth=0.8, linestyle='--', alpha=0.5)
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.set_xlabel('Horizon (quarters)', fontsize=11)
    if idx == 0:
        ax.set_ylabel('Response', fontsize=11)
    ax.grid(True, alpha=0.3)

fig.suptitle(
    'Energy Price Transmission: Responses to Oil Price Shock',
    fontsize=14, fontweight='bold', y=1.03,
)
fig.tight_layout()
fig.savefig(f'{FIGURE_DIR}/energy_price_transmission.png', dpi=150, bbox_inches='tight')
plt.show()

# Economic interpretation
print('\nEnergy Price Transmission Summary')
print('=' * 55)
for var in ['gas_price', 'electricity_price']:
    vals = irf_energy[var, 'oil_price']
    peak_idx = np.argmax(np.abs(vals))
    print(f'\n  {var} response to oil shock:')
    print(f'    Impact (h=0): {vals[0]:.4f}')
    print(f'    Peak:         {vals[peak_idx]:.4f} at h={peak_idx}')
    print(f'    Long-run:     {vals[-1]:.4f}')

---

## Section 7: Diagnostics and Best Practices (10 min)

### 7.1 Pre-IRF Checklist

Before interpreting IRFs, verify:

1. **Stability**: All eigenvalues of the companion matrix lie strictly inside the unit circle
2. **Lag order**: Selected using information criteria (AIC, BIC) -- too few lags bias IRFs, too many inflate variance
3. **Residual diagnostics**: No severe autocorrelation or heteroscedasticity
4. **Stationarity**: Variables should be stationary (or cointegrated) for valid inference

### 7.2 Common Pitfalls

| Pitfall | Consequence | Solution |
|:--------|:-----------|:---------|
| Unstable VAR | IRFs explode | Check eigenvalues, reduce lags, difference data |
| Wrong lag order | Biased IRFs | Use information criteria, try multiple lag orders |
| Unjustified Cholesky ordering | Misleading structural interpretation | Test robustness to ordering, or use GIRFs |
| Too few bootstrap reps | Noisy confidence intervals | Use B >= 500 for final results |
| Ignoring unit roots | Spurious IRFs | Test for stationarity, use VECM if needed |
| Over-interpreting long-horizon IRFs | Uncertainty is large | Focus on short/medium horizon results |

### 7.3 Robustness: Testing Different Cholesky Orderings

In [ ]:
# Robustness check: compare 3 different orderings
orderings = {
    'Standard': ['interest_rate', 'inflation', 'gdp_growth'],
    'Alternative 1': ['inflation', 'interest_rate', 'gdp_growth'],
    'Alternative 2': ['gdp_growth', 'inflation', 'interest_rate'],
}

fig, ax = plt.subplots(figsize=(10, 6))
colors_ord = ['#2166ac', '#b2182b', '#4dac26']
styles = ['-', '--', '-.']
h = np.arange(21)

for idx, (label, order) in enumerate(orderings.items()):
    irf_ord = results.irf(
        periods=20,
        method='cholesky',
        order=order,
    )
    vals = irf_ord['gdp_growth', 'interest_rate']
    ax.plot(
        h, vals,
        color=colors_ord[idx], linewidth=2,
        linestyle=styles[idx], label=f'{label}: {" -> ".join(order)}',
    )

ax.axhline(y=0, color='black', linewidth=0.8, linestyle='--', alpha=0.5)
ax.set_xlabel('Horizon (quarters)', fontsize=12)
ax.set_ylabel('Response of GDP Growth', fontsize=12)
ax.set_title(
    'Robustness: GDP Response to Interest Rate Shock\nacross Cholesky Orderings',
    fontsize=14, fontweight='bold',
)
ax.legend(fontsize=9, loc='best')
ax.grid(True, alpha=0.3)
fig.tight_layout()
fig.savefig(f'{FIGURE_DIR}/ordering_robustness.png', dpi=150, bbox_inches='tight')
plt.show()

print('\n--- Key Insight ---')
print('If IRFs are robust to ordering, the results are more credible.')
print('Large differences across orderings suggest the identification is fragile.')
print('In that case, consider using Generalized IRFs instead.')

### 7.4 Reporting Standards

When presenting IRFs in academic or policy work:

1. **Always report confidence intervals** (preferably bootstrap with B >= 500)
2. **State the identification scheme** (Cholesky ordering, sign restrictions, etc.)
3. **Justify the variable ordering** from economic theory
4. **Include robustness checks** (alternative orderings, lag orders, sample periods)
5. **Report the number of bootstrap replications** and the confidence level
6. **Show both level and cumulative IRFs** when variables are in growth rates
7. **Label axes clearly**: "horizon" on x-axis, "response of [variable]" on y-axis
8. **Discuss economic significance** alongside statistical significance

---

## Summary

### Key Takeaways

1. **IRFs trace the dynamic effects** of structural shocks through the VAR system. They are derived from the Moving Average representation of the VAR.

2. **The identification problem** arises because reduced-form residuals are correlated. Orthogonalization is needed to give IRFs a structural interpretation.

3. **Cholesky IRFs** assume a recursive causal structure determined by variable ordering. The ordering must be justified by economic theory.

4. **Generalized IRFs** (Pesaran-Shin, 1998) are order-invariant and useful when no clear causal ordering exists, but they lack a structural interpretation.

5. **Cumulative IRFs** measure the total accumulated effect and are essential when variables are in growth rates (to recover level effects).

6. **Bootstrap confidence intervals** are critical for inference. Always report CIs and check whether responses are statistically distinguishable from zero.

### Interpretation Guidelines

| Aspect | What to look for |
|:-------|:----------------|
| **Sign** | Does the response match economic theory? |
| **Magnitude** | How large is the effect in economic terms? |
| **Timing** | When does the peak effect occur? |
| **Persistence** | How quickly does the effect dissipate? |
| **Significance** | Is the confidence interval away from zero? |
| **Robustness** | Is the result stable across orderings/specifications? |

### Next Steps

- **Notebook 03**: Forecast Error Variance Decomposition (FEVD) -- how much of the forecast uncertainty in each variable is due to shocks in other variables
- **Notebook 04**: Granger Causality and Forecasting -- testing predictive relationships and out-of-sample forecasting

### References

- Christiano, L. J., Eichenbaum, M., & Evans, C. L. (1999). Monetary policy shocks: What have we learned and to what end? *Handbook of Macroeconomics*, 1, 65-148.
- Kilian, L. (1998). Small-sample confidence intervals for impulse response functions. *Review of Economics and Statistics*, 80(2), 218-230.
- Lutkepohl, H. (2005). *New Introduction to Multiple Time Series Analysis*. Springer.
- Pesaran, H. H., & Shin, Y. (1998). Generalized impulse response analysis in linear multivariate models. *Economics Letters*, 58(1), 17-29.
- Sims, C. A. (1980). Macroeconomics and reality. *Econometrica*, 48(1), 1-48.

---

## Exercises

### Exercise 1: Ordering Sensitivity (Medium)

Using the macro panel data:

1. Compute Cholesky IRFs with at least **3 different orderings** of the variables `['interest_rate', 'inflation', 'gdp_growth']`
2. Plot the **inflation response to a GDP growth shock** for all 3 orderings on the same chart
3. Compare the results and discuss: How sensitive is this particular IRF to the ordering? What does this tell you about the contemporaneous relationship between these variables?
4. Compute the Generalized IRF for the same impulse-response pair and overlay it on the plot. How does it compare?

In [ ]:
# YOUR CODE HERE


### Exercise 2: Bootstrap CI Analysis (Easy)

Using the macro panel VAR:

1. Compute bootstrap IRFs with `n_bootstrap=200` and `ci_level=0.90` (instead of 0.95)
2. Plot the GDP response to an interest rate shock with the 90% CI
3. At which horizons is the response statistically significant at the 90% level?
4. Compare with the 95% CI results from Section 5: does changing the confidence level affect the significance conclusions?

In [ ]:
# YOUR CODE HERE


### Exercise 3: Peak Effect Identification (Medium)

For all 9 impulse-response pairs (3 shocks x 3 responses) in the macro VAR:

1. Compute Cholesky IRFs with the standard ordering `['interest_rate', 'inflation', 'gdp_growth']`
2. For each pair, identify:
   - The **horizon of the peak (absolute) effect**
   - The **magnitude of the peak effect**
   - Whether the peak is positive or negative
3. Present the results in a summary table (pandas DataFrame)
4. Which shock has the largest absolute peak effect on GDP growth? At what horizon?

In [ ]:
# YOUR CODE HERE


### Exercise 4: Multiplier Calculation (Hard)

Using both level and cumulative IRFs from the macro VAR:

1. Compute the **short-run multiplier** (cumulative IRF at h=4) for all 3 variables in response to an interest rate shock
2. Compute the **medium-run multiplier** (h=8) and **long-run multiplier** (h=20)
3. Verify the cumulative relationship: confirm that `cumulative_irf[h] = sum(level_irf[0:h+1])` for all horizons
4. If GDP growth falls by -0.15 percentage points in total (cumulative) in response to a 1 s.d. interest rate shock over 8 quarters, what does that imply for the level of GDP by the end of 2 years? Discuss.
5. Plot a "multiplier timeline" showing the evolution of the cumulative multiplier from h=0 to h=20 for all 3 response variables.

In [ ]:
# YOUR CODE HERE
